### How numbers are encrypted

When a scalar is multiplied by a point on an elliptic curve, another elliptic curve point is produced. That is $P = pG$, where $p$ is a scalar and $G$ is the generator. Given $P$ and $G$, we cannot determine $p$.

Assume $pq = r$ and $P = pG$, $Q = qG$ and $R = rG$, then we want a function $f$ such that:
$f(P,Q) = R$ and $f(P,Q) \not = R$ when $pq \not = r$, $\forall p,q \in G$.

However, this is typically expressed as:
$$
f(P,Q) = f (R, G)
$$
where $G$ is the generator point, and can be thought of as "1" for the pairing operation ($P \times Q = R \times 1$).

### Bilinearity

$$
f(aG, bG)  = f(abG, G) = f(G, abG)
$$

The essential property of bilinear pairings is that if you plug in two points that are multiples of the generator point ($aG$, $bG$), the result is equal to plugging in the product of those two numbers times the generator point ($abG$) and the generator point itself ($G$).

In the literature, what we are calling $f$ is usually written as $e$ as the following:
$$
e: G \times G \rightarrow G_{T}
$$
The subscript $T$ means it is the "target group". To respect the literature, we will write the above as:
$$
e(aG, bG) = e(abG, G) = e(G, abG)
$$

### Symmetric and Asymmetric Groups

The notation above implies that we are using the same generator and elliptic curve groups everywhere when we say:
$$
e(aG, bG) = e(abG, G)
$$
In practice however, it turns out to be easier to create bilinear pairings when using a different group for both of the arguments and the output result. Specifically we say:
$$
e(a, b) \rightarrow c, a \in G, b \in G', c \in G''
$$
None of the groups used are the same.
However, the property we care about still holds:
$$
e(aG, bG') = e(abG, G') = e(G, abG')
$$
Notice how group $G''$ is not explicitly mentioned, but that is the co-domain of $e(G, G')$.

In a symmetric pairing, the same group is used for both arguments. On the other hand, in asymmetric pairing, the arguments use different groups.

### Field Extensions

Bilinear pairings are agnostic to the kind of groups you opt for, but EVM's bilinear pairing uses elliptic curves with _field extensions_.
Usually an EC point is though of as *one* pair $(x,y)$. With field extensions, they consist of several $(x,y)$ pairs.
> Elliptic Curve "points" with more than 2 dimensions.
We usually represent them as vectors:
```solidity
struct G2Point {
    uint256 X[2];
    uint256 Y[2];
}
```

While it might seems weird at first, it still has the same properties of a cyclic group:
- Closed under addition, which is associative
- An identity element
- Each element is invertible
- There is a generator

Notice that the bilinearity property is hard to get. The chances that $e(aG, bG') = e(abG, G') = e(G, abG')$ is true for three random groups are pretty slim.

---

### Bilinear Pairings in Python

In [4]:
from py_ecc.bn128 import G1, G2, pairing, add, multiply, eq
print(G1)
print(G2)

(1, 2)
((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))


Testing some identities:

In [7]:
print(eq(add(G1, G1), multiply(G1, 2)))
print(eq(add(G2, G2), multiply(G2, 2)))

True
True


Now let's go to the good part: pairing!

In [12]:
A = multiply(G2, 5)
B = multiply(G1, 6)
print(pairing(A, B))

(2737733771970589720147436295258995541017562764748775046990018238171083065584, 7355949162177082646197064865377481127039528955264110892670278171102027012957, 1389120597320745437757553030085914762401499323567753964656133081964131780715, 4070774491543958907062047566637569178763974576144707726129772744684275725184, 10823414137019623021013733227099721415368303324105358213304652659949682568395, 12697986880222911287030392175914090722292212037466224705879408804162602333706, 17697943997237703208660786428217562403504798830995307420075922564993565300645, 2702065915136914071855531840006964465333491722231468583849464437921405019853, 6762652910450025398171695126080749677225757293012137750262928324249233167133, 9495821522287762858490254871883860235240788822777455638443279749602676973720, 17813117134675140440034537765301248350834713246854720915775731738875700896539, 21027635025043266481235488683404016989778194881701554135606154029160033599034)


Thankfully, we have need to see the result of `pairing(A, B)`, we can simply do the following:

In [14]:
A = multiply(G2, 5)
B = multiply(G1, 6)
C = multiply(G2, 5 * 6)

pairing(A, B) == pairing(C, G1)

True

So if we want to be mathematical about this, our bn128 pairing does the following:
$$
e: G1 \times G2 \rightarrow G12
$$

### Bilinear Pairings in Ethereum

The `py_ecc` library is maintained by the Ethereum Foundation and it powers the precompile at address`0x8` in the PyEVM implementation.

The Ethereum Precompile defined in [EIP-197](https://eips.ethereum.org/EIPS/eip-197) works on points in $G1$ and $G2$ and _implicitly_ works on points in $G12$.

The specification of this precompile will seem a little weird at first. It takes a list of $G1$ and $G2$ points, laid out as follows:

$$
A_1B_1A_2B_2 \ldots A_nB_n
$$

These were originally created as:
$$
A_1 = a_1G1 \\
B_1 = b_1G2 \\
A_2 = a_2G1 \\
B_2 = b_2G2 \\
\ldots \\
A_n = a_nG1 \\
B_n = b_nG2
$$

The precompile returns $1$ if the following is true:
$$
a_1b_1 + a_2b_2 + \ldots + a_nb_n = 0
$$
and $0$ otherwise.

The reasons to have that instead of a general wrapper around $e: G1 \times G2 \rightarrow G12$ are:
1. $G12$ points are huge (12 pairs of coordinates = $12 \times 2 \times 128 = 3072$ bits), which would be very expensive to store or process at the EVM level.
2. Most ZK verification algorithms do not check for the value of the output of a pairing, but only that it is equal to the sum of other pairings.

For instance, the final step in `groth16`  (the ZK algorithm used by Tornado Cash) looks like the following:
$$
e(A_1, B_2) = e(\alpha_{1}, \beta_{2}) + e(L_{1}, \gamma_{2}) + e(C_{1}, \delta_{2})
$$
WHere each variable is a point in either $G1$ or $G2$ based on its subscript notation.
If we manipulate the expression above, we can write it as:
$$
0 = e(-A_{1}, B_{2}) + e(\alpha_{1}, \beta_{2}) + e(L_{1}, \gamma_{2}) + e(C_{1}, \delta_{2})
$$
which matches the precompile.

If we look at the [verification code](https://github.com/tornadocash/tornado-core/blob/master/contracts/Verifier.sol#L192-L230) from Tornado Cash, we can see it implements exactly this (even the greek letters match!):

```solidity
Pairing.pairing(
    Pairing.negate(_proof.A), // -A
    _proof.B, 
    vk.alfa1,
    vk.beta2,
    vk_x,
    vk.gamma2,
    _proof.C,
    vk.delta2
  );
}
```

Notice that `proof.A`, `proof.B` and `proof.C` derived from the `proof` param and `vk_x` is computed from the `input` param.

### Sum of preimages
The key insight here is that if:
$$
ab + cd = 0
$$
Then it must also be true that:
$$
A_1B_2 + C_1D_2 = 0_{12} \hspace{0.5cm} A_1,C_1 \in G1, B_2, D_2, \in G2
$$
in $G12$ space.

The sum of pairings is zero if and only if the sum of the products of the discrete logarithms is zero.

### Homework assignment

In [30]:
from py_ecc.bn128 import neg, add, multiply, G1, G2, final_exponentiate, FQ12, pairing

(G2_x1, G2_y1) = G2[0].coeffs
(G2_x2, G2_y2) = G2[1].coeffs
print(G2)
print("G2 = G2Point([uint256(%d), uint256(%d)], [uint256(%d), uint256(%d)])"%(G2_x1, G2_x2, G2_y1, G2_y2))


alpha = 1
beta = 2
gamma = 3
delta = 4

c = 8
x1 = 5
x2 = 6
x3 = 7
xTotal = x1 + x2 + x3

# negate G1 * a to make the equation sum up to 0

alpha1 = multiply(G1, alpha)
beta2 = multiply(G2, beta)
print(beta2)
gamma2 = multiply(G2, gamma)
delta2 = multiply(G2, delta)

(aG1_x, aG1_y) = alpha1
(bG2_x1, bG2_y1) = beta2[0].coeffs
(bG2_x2, bG2_y2) = beta2[1].coeffs
(cG2_x1, cG2_y1) = gamma2[0].coeffs
(cG2_x2, cG2_y2) = gamma2[1].coeffs
(dG2_x1, dG2_y1) = delta2[0].coeffs
(dG2_x2, dG2_y2) = delta2[1].coeffs

print("alpha1 = G1Point(%d, %d)"%(aG1_x, aG1_y))
print("beta2 = G2Point([uint256(%d), uint256(%d)], [uint256(%d), uint256(%d)])"%(bG2_x1, bG2_x2, bG2_y1, bG2_y2))
print("gamma2 = G2Point([uint256(%d), uint256(%d)], [uint256(%d), uint256(%d)])"%(cG2_x1, cG2_x2, cG2_y1, cG2_y2))
print("delta2 = G2Point([uint256(%d), uint256(%d)], [uint256(%d), uint256(%d)])"%(dG2_x1, dG2_x2, dG2_y1, dG2_y2))

# We know that
# assert a * b == (alpha * beta) + (xTotal * gamma) + (c * delta)
# where xTotal = x1 + x2 + x3

rhs = (alpha * beta + (xTotal * gamma) + (c * delta))
print(rhs)

## a = 227
## b = 8

def validatePairing(A1, B2, C1, x1, x2, x3):
    Z1 = add(add(multiply(G1, x1), multiply(G1, x2)), multiply(G1, x3))
    assert(Z1 == multiply(G1, x1 + x2 + x3))
    # use final_exponentiate to pair
    assert final_exponentiate(pairing(B2, neg(A1)) * pairing(beta2, alpha1) * pairing(gamma2, Z1) * pairing(delta2, C1)) == FQ12.one()

a = 11
b = 8

assert 0 == -a*b + alpha*beta + (x1 + x2 + x3)*gamma + c * delta

A1 = multiply(G1, a)
B2 = multiply(G2, b)
C1 = multiply(G1, c)

# validatePairing(A1, B2, C1, x1, x2, x3)

(B2_x1, B2_y1) = B2[0].coeffs
(B2_x2, B2_y2) = B2[1].coeffs

print("A1 = G1Point(%d, %d)"%A1)
print("B2 = G2Point([uint256(%d), uint256(%d)], [uint256(%d), uint256(%d)])"%(B2_x1, B2_x2, B2_y1, B2_y2))
print("C1 = G1Point(%d, %d)"%C1)
print("x1 = ", x1)
print("x2 = ", x2)
print("x3 = ", x3)

((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))
G2 = G2Point([uint256(10857046999023057135944570762232829481370756359578518086990519993285655852781), uint256(8495653923123431417604973247489272438418190587263600148770280649306958101930)], [uint256(11559732032986387107991004021392285783925812861821192530917403151452391805634), uint256(4082367875863433681332203403145435568316851327593401208105741076214120093531)])
((18029695676650738226693292988307914797657423701064905010927197838374790804409, 14583779054894525174450323658765874724019480979794335525732096752006891875705), (2140229616977736810657479771656733941598412651537078903776637920509952744750, 11474861747383700316476719153975578001603231366361248090558603872215261634